In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import string
from os import listdir
import os
import errno
import subprocess
from collections import defaultdict
import operator
import re
import pickle
import six # needed for Google Cloud client


In [2]:
!pip install unidecode
import unidecode

In [3]:
!pip install --upgrade --ignore-installed google-cloud-language
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

  Using cached google_cloud_language-0.27.0-py2.py3-none-any.whl
  Using cached enum34-1.1.6-py2-none-any.whl
  Using cached google_cloud_core-0.26.0-py2.py3-none-any.whl
  Using cached google_auth-1.0.2-py2.py3-none-any.whl
  Using cached protobuf-3.3.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached requests-2.18.3-py2.py3-none-any.whl
  Using cached grpcio-1.4.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached six-1.10.0-py2.py3-none-any.whl
  Using cached rsa-3.4.2-py2.py3-none-any.whl
  Using cached cachetools-2.0.1-py2.py3-none-any.whl
  Using cached pyasn1_modules-0.0.11-py2.py3-none-any.whl
  Using cached pyasn1-0.3.2-py2.py3-none-any.whl
  Using cached setuptools-36.2.7-py2.py3-none-any.whl
  Using cached urllib3-1.22-py2.py3-none-any.whl
  Using cached idna-2.5-py2.py3-none-any.whl
  Using cached certifi-2017.7.27.1-py2.py3-none-any.whl
  Using cached chardet-3.0.4-py2.py3-none-any.whl
  Using cached futures-3.1.1-py2-none-any.whl
  Using cached monotonic-1.3-py2.py3-none-an

In [4]:
!ls -lr recipes

total 182236
-rw-rw-r-- 1 apanza apanza  6064790 Aug 13 01:01 pasta-and-noodles.json
-rw-rw-r-- 1 apanza apanza 22297686 Aug 12 20:18 meat-and-poultry.json
-rw-rw-r-- 1 apanza apanza 22103562 Aug 12 06:09 main-dish.json
-rw-rw-r-- 1 apanza apanza  1239579 Aug 11 17:47 lunch.json
-rw-rw-r-- 1 apanza apanza 17783780 Aug 11 17:01 holidays-and-events.json
-rw-rw-r-- 1 apanza apanza  5211522 Aug 11 04:49 healthy-recipes.json
-rw-rw-r-- 1 apanza apanza 33283609 Aug 11 00:26 fruits-and-vegetables.json
-rw-rw-r-- 1 apanza apanza 40589000 Aug 10 06:50 everyday-cooking.json
-rw-rw-r-- 1 apanza apanza 38022625 Aug 12 02:55 allRecipes_tony_categories.json


In [5]:
recipe_dirname = 'recipes'
try:
    os.makedirs(recipe_dirname)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [6]:
datafiles = [
    'everyday-cooking.json',
    'fruits-and-vegetables.json',
    'pasta-and-noodles.json',
    'allRecipes_tony_categories.json',
    'main-dish.json',
    'meat-and-poultry.json',
    'lunch.json',
    'healthy-recipes.json',
    'holidays-and-events.json',
]

for filename in datafiles:
    if not os.path.isfile(os.path.join(recipe_dirname, filename)):
        print "Need to download: {}".format(os.path.join(recipe_dirname, filename))
        # TODO: use boto3 to download file
        subprocess.call(['aws', 's3', 'cp', 's3://RecipeVectors/'+filename, recipe_dirname])
    else:
        print "datafile already present: {}".format(os.path.join(recipe_dirname, filename))

datafile already present: recipes/everyday-cooking.json
datafile already present: recipes/fruits-and-vegetables.json
datafile already present: recipes/pasta-and-noodles.json
datafile already present: recipes/allRecipes_tony_categories.json
datafile already present: recipes/main-dish.json
datafile already present: recipes/meat-and-poultry.json
datafile already present: recipes/lunch.json
datafile already present: recipes/healthy-recipes.json
datafile already present: recipes/holidays-and-events.json


In [7]:
df =pd.DataFrame()
cnt=0
for file in listdir('recipes'):
#    print 'recipes/'+file
    df = df.append(pd.read_json('recipes/'+file, dtype={}))
    print 'Loaded {0} recipes from {1}'.format(len(df)-cnt,file)
    cnt=len(df)
print 'Total {0}'.format(len(df))
df.sample(5)

Loaded 29453 recipes from everyday-cooking.json
Loaded 24107 recipes from fruits-and-vegetables.json
Loaded 3969 recipes from pasta-and-noodles.json
Loaded 28877 recipes from allRecipes_tony_categories.json
Loaded 14456 recipes from main-dish.json
Loaded 14729 recipes from meat-and-poultry.json
Loaded 880 recipes from lunch.json
Loaded 4235 recipes from healthy-recipes.json
Loaded 12679 recipes from holidays-and-events.json
Total 133385


,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url
9229,"[Cranberry Desserts, Fruit Desserts, Holiday C...",NaN,NaN,"[1 cup chopped golden raisins, 1/2 teaspoon sa...",[Mix sugar substitute with orange juice. Pour ...,Sugarless Fruitcake,4.67,3,http://allrecipes.com/recipe/9282/sugarless-fr...
2848,"[Strawberry Pie, Fruit Pies, Pie Recipes, Stra...",PT1H,NaN,[1 recipe pastry for a 9 inch double crust pie...,[Preheat oven to 425 degrees F (220 degrees C....,Most Spectacular Strawberry Pie,3.88,24,http://allrecipes.com/recipe/20249/most-specta...
4881,"[Birds Eye®, Trusted Brands, Pan Fried Chicken...",PT25M,NaN,[1 pound boneless skinless chicken breast halv...,[Toss chicken with 1 tablespoon olive oil and ...,Garlic Chicken and Zucchini,4.24,31,http://allrecipes.com/recipe/233021/garlic-chi...
9343,"[Chicken Sandwiches, Sandwiches, Everyday Cook...",PT1H50M,NaN,"[1 (2 to 3 pound) whole chicken, 4 tablespoons...",[Preheat oven to 350 degrees F (175 degrees C)...,Roasted Chicken Salad Bites,4.77,38,http://allrecipes.com/recipe/26926/roasted-chi...
28254,"[Trusted Brands, Breakfast Drinks, Breakfast a...",PT5M,NaN,"[1 cup strawberry kiwi juice, 1 packet Strawbe...","[Place juice, Carnation Breakfast Essentials D...",Berry Smoothie from Carnation Breakfast Essent...,0.00,0,http://allrecipes.com/recipe/256895/berry-smoo...


In [8]:

categories = defaultdict(int)
for idx, row in df.iterrows():
    # print type(row['categories'])
    if  type(row['categories']) != float:
        for cat in row['categories']:
            categories[cat] += 1 
print 'Top 30 Categories:'
for item in sorted(categories.items(), reverse=True, key=operator.itemgetter(1))[:30]:
    print item

Top 30 Categories:
(u'Recipes', 130426)
(u'Everyday Cooking', 101321)
(u'Main Dishes', 48355)
(u'Desserts', 26288)
(u'Appetizers and Snacks', 12636)
(u'Pasta by Shape', 10978)
(u'Chicken Breasts', 9792)
(u'Side Dishes', 9275)
(u'Soups, Stews and Chili', 8361)
(u'Cookies', 7791)
(u'Fruit Desserts', 7737)
(u'Pasta Main Dishes', 7491)
(u'Salad Recipes', 7424)
(u'Bread Recipes', 7294)
(u'Cake Recipes', 7006)
(u'Breakfast and Brunch', 6862)
(u'Drinks', 6822)
(u'Vanilla Extract', 6737)
(u'Italian Recipes', 6717)
(u'Faceless Recipes', 6703)
(u'Asian Recipes', 6697)
(u'Trusted Brands', 6075)
(u'Mexican Recipes', 5537)
(u'Soup', 5301)
(u'Chocolate Desserts', 5197)
(u'Dips and Spreads', 4903)
(u'Baked and Roasted Chicken', 4482)
(u'Vegan Recipes', 4478)
(u'Cheese Appetizers', 4349)
(u'Fish Recipes', 4319)


In [9]:

def convertCookingTimeToMinutes(text):
    if type(text) == float:
        return None
    text = text.replace('PT','')
    arr = re.findall(r'[0-9]+', text)
    minute = 0
    if len(arr) == 2:
        minute = int(arr[0])* 60 + int(arr[1])
    if len(arr) == 1:
        if 'M' in text:
            minute = int(arr[0])
        if 'H' in text:
            minute = int(arr[0]) * 60
    return minute
df['cookingTimeMinutes'] = df['cookingTime'].apply(convertCookingTimeToMinutes)
df[['cookingTime','cookingTimeMinutes']].sample(5)

,cookingTime,cookingTimeMinutes
480,PT5M,5.0
5688,PT1H,60.0
12958,PT5M,5.0
26301,NaN,NaN
25877,PT3H25M,205.0


In [10]:
df.to_pickle("original_recipe_data.pkl")  # where to save it, usually as a .pkl
# aws s3 cp original_recipe_data.pkl s3://RecipeVectors

In [11]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()&*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        translate_table = dict((ord(char), unicode(translate_to))
                               for char in not_letters)
    else:
        assert isinstance(to_translate, str)
        translate_table = string.maketrans(not_letters,
                                           translate_to
                                              *len(not_letters))
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl', 'fluid',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        ###
        'loaf', 'loaves',
        'pouch', 'pouches',
        'wedge', 'wedges',
        'drop', 'drops',
        'amount', 'amounts',
        'bulk', 'bulks',
        'coating', 'coatings',
        'carton', 'cartons',
        'count',
        'dusting', 'dustings',
        'roll', 'rolls',
        'hint', 'hints',
        'round', 'rounds',
        'cube', 'cubes',
        'husk', 'husks',
        'envelope', 'envelopes',
        'container', 'containers',
        'dash', 'dashes',
        'bitesize', 'bitesized',
        'bite', 'sized',
        'size',
        'each',
        'taste', 'desired',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'tub', 'tubs',
        'slab', 'slabs',
        'sprig', 'sprigs',
        'stalk', 'stalks',
        'matchstick', 'matchsticks',
        'balls',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'packet', 'packets',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
        'bag', 'bags',
        'dash', 'dashes',
        'container', 'containers',
        'envelope', 'envelopes',
        'rounds',
        'sheet', 'sheets',
        'squares',
        'whole',
        'extra', 'extras',
        'dozen', 'dozens',
        'half', 'halves',
        'third', 'thirds',
        'quarter', 'quarters',
        'fifth', 'fifths',
        'eighth', 'eighths',
    ]
    
    preparatory_descriptions = [
        'baked',
        'beat', 'beaten', 'wellbeaten',
        'blanched',
        'blended',
        'boiled',
        'bottled',
        'broiled',
        'broken',
        'browned',
        'canned',
        'chilled',
        'chipped',
        'chopped',
        'cleaned',
        'converted',
        'cooked',
        'cooled',
        'cored',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'defrosted',
        'deshelled',
        'desilked',
        'deveined',
        'diced', 'dice',
        'dissolved',
        'divided',
        'drained',
        'dried',
        'filleted',
        'filtered',
        'flaked',
        'fresh',
        'fried',
        'frozen',
        'grated',
        'grilled',
        'ground',
        'halved',
        'hardened',
        'heated',
        'hulled',
        'husked',
        'jarred',
        'juiced',
        'julienned', 'julienne', 'juliennecut', 'juliennesliced',
        'kneaded', 'kneading',
        'marinated',
        'mashed',
        'matchstickcut',
        'melted',
        'microwaved',
        'minced',
        'mixed',
        'opened',
        'packed',
        'peeled',
        'pitted',
        'prepared',
        'pressed',
        'prewashed',
        'processed',
        'quartered',
        'raw',
        'refrigerated',
        'removed',
        'rinsed',
        'ripe',
        'ripened',
        'ripped',
        'roasted',
        'rolled', 'rolling',
        'salted',
        'saved',
        'scored',
        'scrubbed',
        'seasoned',
        'sectioned',
        'seeded',
        'separated',
        'shaved',
        'shelled',
        'shredded',
        'shucked',
        'sifted',
        'smashed',
        'smoked',
        'snipped',
        'sliced',
        'slivered',
        'softened',
        'split',
        'squeezed',
        'steamed',
        'stemmed',
        'soaked',
        'sweetened',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'undrained',
        'unhusked',
        'unopened',
        'unpeeled',
        'unsalted',
        'unsweetened',
        'unwashed',
        'use', 'used',
        'warmed',
        'washed',
        'zested',
    ]
    
    other_words_to_filter = [
        # articles:
        'a', 'an', 'the',
        # adverbs:
        'all',
        'approximately',
        'coarsely',
        'crosswise',
        'diagonally', 
        'finely',
        'freshly',
        'horizontally',
        'left',
        'lengthwise', 
        'lightly',
        'like',
        'overnight',
        'piecewise',
        'plus',
        'right',
        'roughly',
        'sideways',
        'slightly',
        'stiffly',
        'then',
        'thickly',
        'thinly',
        'vertically',
        'very',
        'widthwise', 
        # adjectives:
        'according',
        'another',
        'any',
        'certified',
        'classic',
        'coarse', 
        'cold',
        'discarded',
        'dry',
        'extralarge',
        'extrasmall', 
        'fine', 
        'firm',
        'fresh',
        'hard',
        'hot',
        'jumbo',
        'large', 
        'lean',
        'leftover',
        'less',
        'long',
        'medium',
        'more',
        'multicolored',
        'natural',
        'needed',
        'only',
        'optional',
        'organic',
        'original',
        'other',
        'pure',
        'real',
        'rough',
        'short',
        'small',
        'stale',
        'soft',
        'such',
        'thick',
        'thickcut',
        'thicklycut',
        'thicklysliced',
        'thicksliced',
        'thin',
        'thincut',
        'thinlycut',
        'thinlysliced',
        'thinsliced',
        # prepositions:
        'about',
        'across',
        'against',
        'apart',
        'as',
        'at',
        'by',
        'for',
        'from',
        'in',
        'into',
        'of',
        'on',
        'to',
        'with',
        'without',
        # conjunctions:
        'and',
        'if',
        'or',
        # abbreviations:
        'eg', 'ie',
        # nouns:
        'directions',
        'hand',
        'ingredients',
        'list',
        'room',
        'temperature',
        # verbs:
        'add',
        'including',
    ]
    
    #print "Before..."
    #print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    #print cleaned_ingred_list
    
    # split on whitespace
    # throw out meas units
    #print
    #print "After..."
    ingreds_only = [' '.join(w for w in unidecode.unidecode(ingred_item).split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
         for ingred_item in cleaned_ingred_list
         ]
    ingreds_only = filter(None, ingreds_only) # remove empty strings at end
    #print ingreds_only
    
    brand_names = [
        'a(r)',
        'accent(r)',
        'adams(r)',
        'al fresco(r)',
        'accent(r)',
        'annies lemon chive dressing(r)',
        'archer farms(tm)',
        'backdraft fire sauce(r)',
        'barilla(r)',
        'beau monde (tm)',
        'beechers(r) flagship', 'beechers(r)',
        'beer shiner(r)',
        'best foods(r)',
        'better than bouillon(r)',
        'birds eye(r)',
        'bisquick(r)',
        'bisquick (r)',
        'borden(r)',
        'bragg(r)',
        'briannas home style(r)', 'briannas(r)',
        'brooks(r)',
        'bruces(r)',
        'budweiser(r)',
        'buitoni(r)',
        'bumble bee(r)',
        'bushs best(r)', 'bushs homestyle(r)', 'bushs chili magic(r)', 'bushs(r)',
        'butterball(r)',
        'butter buds(r)',
        'campari(r)',
        'campbells(r) healthy request', 'campbells(r)',
        'carolina pride(r)',
        'carroll shelbys(r)',
        'catelli bistro(r)',
        'cavenders(r)',
        'chambord(r)',
        'chef paul prudhommes salmon magic(r)',
        'cheese whiz(r)', 'cheez whiz(r)',
        'cholula(r)',
        'clamato(r)',
        'classicmac(tm)',
        'classico (tm)',
        'classico (r)',
        'classico(r)',
        'claussen(r)',
        'coke(r)',
        'cointreau(r)',
        'college inn(r)',
        'contadina(r)',
        'cool whip(r)',
        'coors(r)',
        'cornnuts (r)',
        'corona(r)',
        'craisins(r)',
        'crazy steves(tm) cajun cukes', 'crazy steves(r)',
        'crisco(r)',
        'daisy(r)',
        'dei fratelli(r)',
        'del monte(r)',
        'diamond crystal(r)',
        'dickinsons(r)',
        'dole(r)',
        'dole asian island crunch(tm)',
        'dole(r) veggie(tm)',
        'doritos(r)',
        'dubliner(r)',
        'dukes(r)',
        'durkee(r)', 'famous sauce(r)',
        'eagle brand(r)',
        'earth balance(r)',
        'eckrich(r)',
        'edward sons(r)',
        'egg beaters(r)',
        'el pato(r)',
        'embasa(r)',
        'emerald(r)',
        'equal(r)',
        'farmland(r)',
        'fat tire(r)',
        'fiesta blend(r)',
        'follow your heart(r) veganaise(r)', 'follow your heart(r)',
        'foster farms(r)',
        'franks(r) redhot', 'franks red hot (r)', 'franks redhot (r)', 'franks redhot(r)', 'redhot(r)', 'franks(r)',
        'frenchs(r)',
        'frichik(r)',
        'fritos(r)',
        'frontera(r)',
        'goya(r)',
        'gebhardt(r)',
        'girards(r) olde venice',
        'good seasons(r)',
        'gourmet garden(tm)',
        'grapenuts(tm)',
        'great american spice co(tm)',
        'green giant(r)',
        'guinness(r)',
        'harp(r)',
        'hatch(r)',
        'healthy choice(r)',
        'heineken(r)', 
        'heinz (r)', 'heinz(r)',
        "hellmann's(r) best foods(r)", 'hellmanns(r) best foods(r)', 'hellmanns(r)best foods(r)',
        'hellmanns light(r)', 'hellmans(r) light', 'hellmanns(r) low fat',
        'hellmans(r)', "hellmann's(r)",
        'hidden valley ranch(r)', 'hidden valley(r) original ranch(r)',
        'hillshire farm(r)',
        'holland house(r)',
        'horizon(r)',
        'hormel(r)', 'hormel(tm)',
        'house tsang(r)',
        'hunts(r)',
        'idahoan(r)','baby reds(r)', 'buttery homestyle(r)',
        'imagine(r)', 'imagine(tm)',
        'jack daniels(r)',
        'jello(r)',
        'jennieo(r)',
        'jiffy(r)',
        'jim beam(r)',
        'jimmy dean(r)',
        'johnsonville(r)',
        'johnnys seasoning salt(r)',
        'kame(r)',
        'karys(r)',
        'kc masterpiece(r)',
        'keebler club(r)', 'keebler(r)',
        'keens(r)',
        'kens steak house(r)',
        'kens steak house lite northern italian dressing(r)',
        'kerrygold(r)',
        'kewpie(r)',
        'king arthur(r)',
        'king oscar(r)',
        'kirkland(r)',
        'kitchen basics(r)',
        'kitchen bouquet(r)',
        'kitchen secrets(r)',
        'klondike gourmet(r)', 'klondike goldust(r)', 'klondike(r)',
        'knorr(r) rice sides(tm)', 'knorr(r) fiesta sides(tm)', 'knorr(r)', 'knorrs(r)',
        'knox (r)',
        'kraft(r)',
        'laphroiag(r)',
        'laughing cow(r)',
        'lawrys(r)',
        'libbys(r)',
        'lightlife(r) organic smoky tempeh strips(r)', 'lightlife(r)',
        'lipton(r) recipe secrets(r)', 'lipton(r)',
        'locatelli(r)',
        'loma linda(r)',
        'lotus(r)',
        'louisiana(r)',
        'maggi(r)',
        'mahatma(r)',
        'maifun(r)',
        'maille(r)',
        'malibu(r)',
        'marie callenders(r)',
        'maries(r)',
        'market pantry(tm)',
        'marzettis(r)',
        'maui(r)',
        'mazola(r)',
        'mccormick chili seasoning mix sodium(r)', 'mccormick(r)',
        'mexene(r)',
        'mexenes(r)',
        'mexicorn(r)',
        'miller lite(r)',
        'mimiccreme(r)',
        'minute(r)',
        'miracle whip(r)',
        'miracle whip(tm)',
        'miracle whip (tm)',
        'miracle whip free(r)',
        'miracle whip light(r)', 'light miracle whip(r)',
        'montreal steak seasoning(r)',
        'morningstar farms(r) grillers recipe crumbles(r)', 'morningstar farms(r) recipe crumbles(r)', 'morningstar farms(r)',
        'morton(r) natures seasons(r)', 'morton(r)',
        'motts(r)',
        'mountain dew(r)',
        'mrs butterworths(r)',
        'mrs dash(r)',
        'muir glen(r)',
        'nabisco(r)',
        'natural goodness(tm)',
        'negra modelo(r)',
        'newcastle(r)',
        'newmans own(r)',
        'newmans own(r) lighten up(r)', 'lighten up(r)',
        'niblets(r)',
        'nissin(r) top ramen',
        'no yolks(r)',
        'ocean spray(r)',
        'old bay(r)', 'old bay (tm)', 'old bay(tm)',
        'old el paso(r)',
        'olive garden(r)',
        'oreida(r) steam n mash(r)', 'oreida(r)', 'ore ida(r)',
        'oreo(r)',
        'ortega(r)',
        'ottogi ramyonsari(r)', 'ottogi(r)',
        'quorn(tm)',
        'pace(r)',
        'pam(r)',
        'pataks(r)',
        'penzeys(r)', 'penzys(r)',
        'pepperidge farm(r)',
        'perdue(r)',
        'pernod(r)',
        'perrins(r)',
        'pillsbury grands(r)',
        'planters(r)',
        'plochmans(r)',
        'progresso(r)',
        'ragu(r) old world style(r)', 'ragu(r)',
        'rapunzel(r)',
        'ready rice(r)',
        'realime(r)',
        'red gold(r)',
        'renees(r)',
        'reynolds wrap(r)', 'reynolds(r)',
        'rice sides(tm)',
        'ronzoni(r)',
        'rotel(r)',
        'saco(r)',
        'sambazon',
        'samuel smith(r)',
        'sargento(r)',
        'sason accent(r)',
        'sb(r) golden curry', 'sb(r)',
        'seasonall(r)',
        'shiner bock(r)',
        'silk(r)',
        'simple crisp(tm)',
        'simply balanced(tm)',
        'simply potatoes(r)',
        'slap ya mama(r)',
        'smithfield(r)',
        'snack factory(r)',
        'snickers(r)',
        'so delicious(r)',
        'southern comfort (r)',
        #'spam(r)',
        'spectrum(r)',
        'spice islands(r)',
        'spike(r)',
        'splenda(r)',
        'sriracha(r)',
        'steam whistle pilsner(r)',
        'sunsweet(r) dnoir(tm)',
        'sutton dodge(tm)',
        'swanson(r) vegetable flavor boost(r)', 'swanson(r)', 'natural goodness(r)',
        'sweet baby rays(r)',
        'sweetn low(r)',
        'tabasco(r)', 'tabasco(tm)',
        'taco bell(r)',
        'tajin(r)',
        'tapatio(r)',
        'tater tots(r)',
        'texas pete(r)', 'texas petess(r)',
        'thai kitchen(r)',
        'three bridges(r)',
        'tony chacheres(r)',
        'top ramen(r)',
        'trader joes(r)',
        'truroots(r)',
        'truvia(r)',
        'uncle bens (r)', 'uncle bens(r)',
        'v(r)', # what remains from "V8"
        'valentina(r)',
        'vegenaise(r)',
        'veggie(tm)',
        #'velveeta(r)',
        'vidalia(r)',
        'vietti(r)',
        'vindaloo sauce maya kaimal(r)', 'vindaloo(r)',
        'voskos(r)',
        'wesson(r)',
        'white lily(r)',
        'wishbone(r)',
        'wondra(r)',
        'yoplait(r)',
        'yuengling(r)',
        'yves veggie cuisine(r)',
        'zatarains(r)',
        'zing zang(r)',
    ]
    
    def remove_brands(ingred_str):
        str_without_brand = ingred_str
        for bn in brand_names:
            str_without_brand = str_without_brand.replace(bn, '')
        return str_without_brand
    
    ingreds_without_brands = [' '.join(remove_brands(ingred_str).strip().split()) for ingred_str in ingreds_only]
    
    return ingreds_without_brands

# def ascii_encode(text):
#     return text.encode("ascii", "ignore")

#extract_ingredients(df[['ingredients']].values[867][0])

In [12]:
ingreds_arr_of_lists = np.squeeze(df[['ingredients']].values)
vect_extract_ingred = np.vectorize(extract_ingredients, otypes=[list])

processed_ingreds_arr_of_lists = vect_extract_ingred(ingreds_arr_of_lists)

#print type(processed_ingreds_arr_of_lists)
print "Processed ingredients from {} recipes".format(len(processed_ingreds_arr_of_lists))
#print processed_ingreds_arr_of_lists.size


Processed ingredients from 133385 recipes


In [13]:
flattened_ingred_list = np.hstack(processed_ingreds_arr_of_lists)
print len(flattened_ingred_list)
#print flattened_ingred_list

unique_vocab = sorted(list(set(flattened_ingred_list)))
print unique_vocab
print "{} unique ingredients found".format(len(unique_vocab))

1248420
['', '(r) cayenne pepper sauce', '(r) condensed cream celery soup', '(r) condensed cream mushroom soup', '(r) sweet chili sauce', 'abalone shell', 'absinthe', 'absinthebased bitters', 'acai berry pulp', 'acai berry sorbet', 'acai juice', 'acai powder', 'acai pulp (r)', 'acaipomegranate juice', 'accents(r) sprouted quinoa trio', 'accents(tm) sprouted rice trio', 'accompaniments', 'acesulfame potassium sweetener', 'achar masala', 'achiote annatto seeds', 'achiote paste', 'achiote powder', 'achiote seed', 'achiote seeds', 'acini de pepe pasta', 'acini di pepe pasta', 'acorn squash', 'acorn squashes', 'act ii(r) fat free popcorn', 'active yeast', 'active yeast bread machine yeast', 'active yeast fleischmanns activedry yeast(r)', 'active yeast fleischmanns rapidrise(r)', 'active yeast fleischmanns(r) rapidrise yeast', 'addins', 'additional butter margarine', 'additional butter sugar garnish', 'additional candy sprinkles', 'additional cheddar cheese', 'additional cocktail sauce dippi

In [14]:
with open('unique_ingredients.pkl', 'wb') as f:
    pickle.dump(unique_vocab, f)
    
#!aws s3 cp unique_ingredients.pkl s3://RecipeVectors

In [15]:
# find ingredients that contain brand names
# these are identifiable from the (r) or (tm) in the names, which unidecode puts there in place of \u00ae or \u2122
# visually inspect this list and consider adding more entries to the brand_names list in cell above, then reprocess
brand_name_ingreds = [s for s in unique_vocab if '(r)' in s or '(tm)' in s]
print len(brand_name_ingreds)
#print brand_name_ingreds
for brand_name in brand_name_ingreds:
    print brand_name

1658
(r) cayenne pepper sauce
(r) condensed cream celery soup
(r) condensed cream mushroom soup
(r) sweet chili sauce
acai pulp (r)
accents(r) sprouted quinoa trio
accents(tm) sprouted rice trio
act ii(r) fat free popcorn
active yeast fleischmanns activedry yeast(r)
active yeast fleischmanns rapidrise(r)
active yeast fleischmanns(r) rapidrise yeast
allpurpose flour blue bird(r)
allwhites(r) egg whites
almond extract watkins(r) almond extract
almond orgeat syrup torani(r)
almondflavored liqueur disaronno(r)
almondflavored syrup torani(r)
amaretto liqueur disaronno(r)
amaro liqueur averno(r)
amaro liqueur hum(r)
amaro liqueur luxardo(r) amaro abano
amber beer honey brown lager(r)
angel food cake mix betty crocker(r)
angostura bitters(r)
aniseflavored liqueur herbsaint(r)
annies(tm) bunny grahams(tm) honey snacks
apple juice concentrate old orchard(r)
apple pie spice mccormicks(r)
apricot fruit filling baker(r) dessert filling
apricot fruit filling solo(r)
apricot nectar kerns(r)
apricot 

In [16]:
flat_ingred_arr = [' '.join(lst) for lst in processed_ingreds_arr_of_lists]

cv = CountVectorizer(vocabulary=unique_vocab)
recipe_ingredient_matrix = cv.fit_transform(flat_ingred_arr)

print type(recipe_ingredient_matrix)
print recipe_ingredient_matrix.size

print recipe_ingredient_matrix

<class 'scipy.sparse.csr.csr_matrix'>
1414438
  (0, 529)	1
  (0, 3365)	1
  (0, 3656)	1
  (0, 5852)	1
  (0, 7754)	1
  (0, 11100)	1
  (0, 12792)	1
  (0, 13116)	1
  (0, 13214)	2
  (0, 13222)	1
  (0, 14047)	2
  (0, 16610)	1
  (0, 18815)	1
  (0, 19370)	1
  (0, 19556)	1
  (0, 20625)	1
  (0, 20972)	1
  (1, 946)	2
  (1, 4692)	1
  (1, 6643)	1
  (1, 7354)	1
  (1, 13116)	1
  (1, 16927)	1
  (1, 18815)	1
  (1, 20222)	1
  :	:
  (133381, 20222)	2
  (133381, 20679)	1
  (133381, 20972)	2
  (133382, 1337)	1
  (133382, 3656)	1
  (133382, 7754)	1
  (133382, 9004)	1
  (133382, 14047)	1
  (133382, 16927)	1
  (133382, 17057)	1
  (133383, 11210)	1
  (133383, 12521)	1
  (133383, 18641)	1
  (133383, 18815)	1
  (133383, 20741)	1
  (133383, 20972)	1
  (133384, 3656)	3
  (133384, 12024)	1
  (133384, 12678)	1
  (133384, 13432)	1
  (133384, 13631)	1
  (133384, 14047)	1
  (133384, 16927)	1
  (133384, 17057)	1
  (133384, 19636)	1


In [17]:
# https://stackoverflow.com/a/8980156/2491761
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

save_sparse_csr("sparse_recipe_ingredient_matrix", recipe_ingredient_matrix)

In [18]:
#!aws s3 cp sparse_recipe_ingredient_matrix.npz s3://RecipeVectors

In [19]:
df['cleanedIngredients'] = df['ingredients'].apply(extract_ingredients)
df.sample(5)

,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url,cookingTimeMinutes,cleanedIngredients
2524,"[Beef Chili, Chili, Soups, Stews and Chili, Ev...",PT5H30M,NaN,"[2 pounds lean ground beef, 1/2 large onion, d...","[In a large skillet over medium heat, brown gr...",Atomic Canuck Chili,4.32,78,http://allrecipes.com/recipe/26582/atomic-canu...,330.0,"[beef, onion, red pepper, garlic powder, peppe..."
3146,"[Veggie Smoothies, Green Smoothies, Banana Smo...",PT5M,NaN,"[1 banana, 4 leaves kale, 1/2 lime, juiced, 1 ...","[Combine banana, kale, lime juice, and honey i...",Super Fresh Smoothie,4.00,1,http://allrecipes.com/recipe/255818/super-fres...,5.0,"[banana, leaves kale, lime, honey]"
28397,"[Shrimp Recipes, Shrimp Scampi, Everyday Cooki...",PT45M,Easy version of this classic with the wonderfu...,"[1 cup butter, 2 tablespoons prepared Dijon-st...",[Preheat oven to 450 degrees F (230 degrees C)...,Shrimp Scampi Bake,4.62,1841,http://allrecipes.com/recipe/25874/shrimp-scam...,45.0,"[butter, dijonstyle mustard, lemon juice, garl..."
8057,"[Fruit Pizza, Specialty Desserts, Strawberry D...",PT45M,NaN,"[1 (18 ounce) package sugar cookie dough, room...",[Preheat oven to 375 degrees F (190 degrees C)...,Strawberry Margarita Pizza,4.50,2,http://allrecipes.com/recipe/235877/strawberry...,45.0,"[sugar cookie dough, milk, instant white choco..."
12325,"[Chocolate Brownies, Brownies, Bar Cookies, Co...",PT20M,NaN,"[1 1/4 cups all-purpose flour, 1 teaspoon baki...","[Mix together flour, baking powder, and salt i...",Brownie Mix in a Jar II,4.50,158,http://allrecipes.com/recipe/10646/brownie-mix...,20.0,"[allpurpose flour, baking powder, salt, cocoa ..."


In [20]:
df.to_pickle('CleanedIngredients.pkl')


In [21]:
!aws s3 cp CleanedIngredients.pkl s3://RecipeVectors

upload: ./CleanedIngredients.pkl to s3://RecipeVectors/CleanedIngredients.pkl


In [22]:
def syntax_text(text):
    """Detects syntax in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    for token in tokens:
        print(u'{}: {}'.format(pos_tag[token.part_of_speech.tag],
                               token.text.content))

In [23]:
syntax_text('three cored and peeled green tomatoes')

NUM: three
VERB: cored
CONJ: and
VERB: peeled
ADJ: green
NOUN: tomatoes


In [ ]:
ingred_list = [u'3 avocados - peeled, pitted, and mashed',
 u'1 lime, juiced',
 u'1 teaspoon salt',
 u'1/2 cup diced onion',
 u'3 tablespoons chopped fresh cilantro',
 u'2 roma (plum) tomatoes, diced',
 u'1 teaspoon minced garlic',
 u'1 pinch ground cayenne pepper (optional)',
 u'Add all ingredients to list',
 u'Add all ingredients to list']

